<a href="https://colab.research.google.com/github/danieldrako/Algoritmos-Clasificacion-de-Texto/blob/main/00EtiquetadoRapido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etiquetado en NLTK

## Pipeline básico para Ingles

In [14]:
#@title Dependencias previas
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize #tokenizador de palabras

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
#@title Etiquetado en una línea ...
text = word_tokenize("And now here I am enjoying today")
nltk.pos_tag(text) #Etiquetado

[('And', 'CC'),
 ('now', 'RB'),
 ('here', 'RB'),
 ('I', 'PRP'),
 ('am', 'VBP'),
 ('enjoying', 'VBG'),
 ('today', 'NN')]

In [16]:
#@title Categoria gramatical de cada etiqueta
nltk.download('tagsets') #Conjunto de metadata que dan significado a las etiquetas 
for tag in ['CC', 'RB', 'PRP', 'VBP', 'VBG', 'NN']:
  print(nltk.help.upenn_tagset(tag))#imprime la convención de las etiquetas

CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
None
RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
None
PRP: pronoun, personal
    hers herself him himself hisself it itself me myself one oneself ours
    ourselves ownself self she thee theirs them themselves they thou thy us
None
VBP: verb, present tense, not 3rd person singular
    predominate wrap resort sue twist spill cure lengthen brush terminate
    appear tend stray glisten obtain comprise detest tease attract
    emphasize mold postpone sever return wag ...
None
VBG: verb, present participle or gerund
    telegraphing stirring focusing angering judging stalling lactating
    hankerin' alleging veering capping approaching traveling besieging
    encrypting interrupting erasing wincing ...
None
N

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [17]:
#@title Palabras homónimas
text = word_tokenize("They do not permit other people to get residence permit")
nltk.pos_tag(text) 

[('They', 'PRP'),
 ('do', 'VBP'),
 ('not', 'RB'),
 ('permit', 'VB'),
 ('other', 'JJ'),
 ('people', 'NNS'),
 ('to', 'TO'),
 ('get', 'VB'),
 ('residence', 'NN'),
 ('permit', 'NN')]

## Etiquetado en Español 

Para el ingles, NLTK tiene tokenizador y etiquetador pre-entrenados por defecto. En cambio, para otros idiomas es preciso entrenarlo previamente. 

* usamos el corpus `cess_esp` https://mailman.uib.no/public/corpora/2007-October/005448.html

* el cual usa una convención de etiquetas gramaticales dada por el grupo EAGLES https://www.cs.upc.edu/~nlp/tools/parole-sp.html

In [18]:
nltk.download('cess_esp')
from nltk.corpus import cess_esp as cess
from nltk import UnigramTagger as ut
from nltk import BigramTagger as bt

[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


In [22]:
#@title Entrenamiendo del tagger por unigramas
cess_sents = cess.tagged_sents()#separación de frases 
fraction = int(len(cess_sents)*90/100) #90% del dataset

uni_tagger = ut(cess_sents[:fraction]) #instancia por unigramas
uni_tagger.evaluate(cess_sents[fraction+1:]) #Evaluacion Es la precision con la cual agregó etiquetas

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


0.8069484240687679

In [23]:
uni_tagger.tag("Yo soy una persona muy amable".split(" "))#tokenizar esta frase por espacios

[('Yo', 'pp1csn00'),
 ('soy', 'vsip1s0'),
 ('una', 'di0fs0'),
 ('persona', 'ncfs000'),
 ('muy', 'rg'),
 ('amable', None)]

In [24]:
#@title Entrenamiento del tagger por bigramas
fraction = int(len(cess_sents)*90/100)
bi_tagger = bt(cess_sents[:fraction])
bi_tagger.evaluate(cess_sents[fraction+1:])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  after removing the cwd from sys.path.


0.1095272206303725

In [25]:
bi_tagger.tag("Yo soy una persona muy amable".split(" "))

[('Yo', 'pp1csn00'),
 ('soy', 'vsip1s0'),
 ('una', None),
 ('persona', None),
 ('muy', None),
 ('amable', None)]

# Etiquetado mejorado con Stanza (StanfordNLP)

**¿Que es Stanza?**

* El grupo de investigacion en NLP de Stanford tenía una suite de librerias que ejecutaban varias tareas de NLP, esta suite se unifico en un solo servicio que llamaron **CoreNLP** con base en codigo java: https://stanfordnlp.github.io/CoreNLP/index.html

* Para python existe **StanfordNLP**: https://stanfordnlp.github.io/stanfordnlp/index.html

* Sin embargo, **StanfordNLP** ha sido deprecado y las nuevas versiones de la suite de NLP reciben mantenimiento bajo el nombre de **Stanza**: https://stanfordnlp.github.io/stanza/

Stanza está construido con componentes de red neuronal de alta precisión que también permiten un entrenamiento y una evaluación eficientes con sus propios datos anotados. Los módulos están construidos sobre la biblioteca de PyTorch . Obtendrá un rendimiento mucho más rápido si ejecuta este sistema en una máquina habilitada para GPU.

In [1]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 5.0 MB/s 
     |████████████████████████████████| 216 kB 23.7 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=9bd6deca9e25dc15babef3d52a89c264eb0aefde422e1bf4cb2831dc2a737c32
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


In [2]:

import stanza
stanza.download('es')

INFO:stanza:Downloading default packages for language: es (Spanish) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [3]:
nlp = stanza.Pipeline('es', processors='tokenize,pos') #('español', processors='tokenizador,posicion de etiquetas')
doc = nlp('yo soy una persona muy amable')#elemento a analizar. Instancia en nlp
'''
usa dos componentes en ancora
Ancora: corpus en español en el cual fue entrenado el bloque de procesamiento
=======================
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
=======================

'''

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


In [4]:
for sentence in doc.sentences:
  for word in sentence.words:
    print(word.text, word.pos)

yo PRON
soy AUX
una DET
persona NOUN
muy ADV
amable ADJ
